In [ ]:
login = 'n.chikunov-13'
year = 1994 + hash(f'{login}') % 23
path_to_file = '/var/lib/airflow/airflow.git/dags/n.chikunov-13/vgsales.csv'

default_args = {
    'owner': 'n.chikunov',
    'depends_on_past': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=10),
    'start_date': datetime(2021, 10, 24),
    'schedule_interval': '30 8 * * *'
}

CHAT_ID = -657046922
try:
    BOT_TOKEN = Variable.get('telegram_secret')
except:
    BOT_TOKEN = ''
    
def send_message(context):
    date = context['ds']
    dag_id = context['dag'].dag_id
    message = f'Huge success! Dag {dag_id} completed on {date}'
    if BOT_TOKEN != '':
        bot = telegram.Bot(token=BOT_TOKEN)
        bot.send_message(chat_id=CHAT_ID, text=message)
    else:
        pass

@dag(default_args=default_args, catchup=False)
def n_chikunov_lesson3():
    @task(retries=5, retry_delay=timedelta(1))
    def n_chikunov_get_data():
        data = pd.read_csv(path_to_file)
        data = data.query('Year == @year')
        return data


    @task()
    def n_chikunov_get_top_game(data):
        top_game = data.sort_values('Global_Sales', ascending=False) \
                       .iloc[0]['Name']
        return top_game


    @task()
    def n_chikunov_get_top_genre_eu(data):
        genre_eu = data.groupby('Genre', as_index=False) \
                       .agg({'EU_Sales': 'sum'}) \
                       .sort_values('EU_Sales', ascending=False)
        genre_eu_sales_max = genre_eu['EU_Sales'].max()
        top_genre_eu = genre_eu.query('EU_Sales == @genre_eu_sales_max')['Genre'].unique()
        return top_genre_eu


    @task()
    def n_chikunov_get_top_platform_na(data):
        platform_na= data.query('NA_Sales > 1') \
                         .groupby('Platform', as_index=False) \
                         .agg({'NA_Sales': 'count'}) \
                         .sort_values('NA_Sales', ascending=False)
        platform_games_na_max = platform_na['NA_Sales'].max()
        top_platform_na = platform_na.query('NA_Sales == @platform_games_na_max')['Platform'].unique()
        return top_platform_na


    @task()
    def n_chikunov_get_top_publisher_jp(data):
        publisher_jp = data.groupby('Publisher', as_index=False) \
                           .agg({'JP_Sales': 'mean'}) \
                           .sort_values('JP_Sales', ascending=False)
        publisher_mean_sales_jp_max = publisher_jp['JP_Sales'].max()
        top_publisher_jp = publisher_jp.query('JP_Sales == @publisher_mean_sales_jp_max')['Publisher'].unique()
        return top_publisher_jp


    @task()
    def n_chikunov_get_games_sales_eu_bet_jp(data):
        count_better_in_eu = data.query('Year == @year and EU_Sales > JP_Sales') \
                                 ['Name'].nunique()
        return count_better_in_eu


    @task(on_success_callback=send_message)
    def n_chikunov_print_data(top_game, top_genre_eu, top_platform_na, top_publisher_jp, count_better_in_eu):

        context = get_current_context()
        date = context['ds']

        print(f'''Game sales data in {year} for {date}:
        Best selling game in the world:
            {top_game}
        Best-selling genre in Europe:
            {', '.join(top_genre_eu)}
        Most popular platform for popular games in North America (over 1 million copies sold):
            {', '.join(top_platform_na)}
        Best Publisher in Japan (Highest Average Game Sales):
            {', '.join(top_publisher_jp)}
        The number of games sold in Europe is better than in Japan:
            {count_better_in_eu}''')
        

    data = n_chikunov_get_data()
    top_game = n_chikunov_get_top_game(data)
    top_genre_eu = n_chikunov_get_top_genre_eu(data)
    top_platform_na = n_chikunov_get_top_platform_na(data)
    top_publisher_jp = n_chikunov_get_top_publisher_jp(data)
    count_better_in_eu = n_chikunov_get_games_sales_eu_bet_jp(data)
    n_chikunov_print_data(top_game, top_genre_eu, top_platform_na, top_publisher_jp, count_better_in_eu)
        
n_chikunov_lesson3 = n_chikunov_lesson3()


In [8]:
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
import os

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context
from airflow.models import Variable

In [5]:
login = 'a.kosheleva-14'
year = 1994 + hash(f'{login}') % 23
path_to_file = '/var/lib/airflow/airflow.git/dags/a.kosheleva-14/vgsales-1.csv'

default_args = {
    'owner': 'a.kosheleva',
    'depends_on_past': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=10),
    'start_date': datetime(2021, 11, 19),
    'schedule_interval': '00 6 * * *'
}

In [ ]:
CHAT_ID = -657046922
try:
    BOT_TOKEN = Variable.get('telegram_secret')
except:
    BOT_TOKEN = ''
    
def send_message(context):
    date = context['ds']
    dag_id = context['dag'].dag_id
    message = f'Huge success! Dag {dag_id} completed on {date}'
    if BOT_TOKEN != '':
        bot = telegram.Bot(token=BOT_TOKEN)
        bot.send_message(chat_id=CHAT_ID, text=message)
    else:
        pass

In [9]:
@dag(default_args=default_args, catchup=False)
def a_kosheleva_lesson3():
    @task(retries=3, retry_delay=timedelta(10))
    def a_kosheleva_get_data():
        data = pd.read_csv(path_to_file)
        data = data.query('Year == @year')
        return data

In [12]:
data = pd.read_csv('/home/jupyter-a.kosheleva-14/vgsales-1.csv')
data = data.query('Year == @year')

In [13]:
data

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
17,18,Grand Theft Auto: San Andreas,PS2,2004.0,Action,Take-Two Interactive,9.43,0.40,0.41,10.57,20.81
47,48,Gran Turismo 4,PS2,2004.0,Racing,Sony Computer Entertainment,3.01,0.01,1.10,7.53,11.66
58,59,Pokemon FireRed/Pokemon LeafGreen,GBA,2004.0,Role-Playing,Nintendo,4.34,2.65,3.15,0.35,10.49
59,60,Super Mario 64,DS,2004.0,Platform,Nintendo,5.08,3.11,1.25,0.98,10.42
79,80,Halo 2,XB,2004.0,Shooter,Microsoft Game Studios,6.82,1.53,0.05,0.08,8.49
...,...,...,...,...,...,...,...,...,...,...,...
16330,16333,Ultraman Fighting Evolution 3,PS2,2004.0,Fighting,Banpresto,0.00,0.00,0.01,0.00,0.01
16362,16365,Powerdrome,XB,2004.0,Racing,Evolved Games,0.01,0.00,0.00,0.00,0.01
16375,16378,Colin McRae Rally 04,PC,2004.0,Racing,Codemasters,0.00,0.01,0.00,0.00,0.01
16445,16448,Wade Hixton's Counter Punch,GBA,2004.0,Sports,"Destination Software, Inc",0.01,0.00,0.00,0.00,0.01


In [38]:
top_game = data.sort_values('Global_Sales', ascending = False) \
    .iloc[0]['Name']

'Grand Theft Auto: San Andreas'

In [60]:
top_genre_sorted = data.groupby('Genre', as_index = False) \
    .agg({'EU_Sales':'sum'}) 
top_genre_loc = top_genre_sorted.sort_values(by='EU_Sales', ascending=False).iloc[0]['EU_Sales']
top_genre = top_genre_sorted[top_genre_sorted['EU_Sales']==top_genre_loc].Genre.values

In [61]:
top_genre

array(['Role-Playing'], dtype=object)

In [65]:
top_platform_NA = data.query('NA_Sales > 1') \
    .groupby('Platform', as_index = False) \
    .agg({'Name':'count'}) 
top_platform_loc = top_platform_NA.sort_values(by='Name', ascending=False).iloc[0]['Name']
top_platform_NA = top_platform_NA[top_platform_NA['Name'] == top_platform_loc].Platform.values

In [66]:
top_platform_NA

array(['PS2'], dtype=object)

In [68]:
top_JP_publisher = data.groupby('Publisher', as_index = False) \
    .agg({'JP_Sales':'sum'}) 
top_JPsales_loc = top_JP_publisher.sort_values('JP_Sales', ascending = False).iloc[0]['JP_Sales']
top_JP_publisher = top_JP_publisher[top_JP_publisher['JP_Sales'] == top_JPsales_loc].Publisher.values

In [69]:
top_JP_publisher

array(['Nintendo'], dtype=object)

In [54]:
eu_more_than_jp = data.query('EU_Sales > JP_Sales')['Name'].nunique()

In [55]:
eu_more_than_jp

428

In [ ]:
login = 'a.kosheleva-14'
year = 1994 + hash(f'{login}') % 23
path_to_file = '/var/lib/airflow/airflow.git/dags/a.kosheleva-14/vgsales-1.csv'

default_args = {
    'owner': 'a.kosheleva',
    'depends_on_past': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=10),
    'start_date': datetime(2021, 11, 19),
    'schedule_interval': '00 6 * * *'
}


@dag(default_args=default_args, catchup=False)
def a_kosheleva_lesson3():
    @task(retries=3, retry_delay=timedelta(10))
    def a_kosheleva_get_data():
        data = pd.read_csv(path_to_file)
        data = data.query('Year == @year')
        return data


    @task()
    def a_kosheleva_top_game(data):
        top_game = data.sort_values('Global_Sales', ascending=False) \
                       .iloc[0]['Name']
        return top_game


    @task()
    def a_kosheleva_top_genre(data):
        top_genre_1 = data.groupby('Genre', as_index = False) \
                                .agg({'EU_Sales':'sum'}) 
        top_genre_loc = top_genre_1.sort_values(by='EU_Sales', ascending=False).iloc[0]['EU_Sales']
        top_genre = top_genre_1[top_genre_1['EU_Sales']==top_genre_loc].Genre.values
        return top_genre


    @task()
    def a_kosheleva_top_platform_in_NA(data):
        top_platform_NA = data.query('NA_Sales > 1') \
                                .groupby('Platform', as_index = False) \
                                .agg({'Name':'count'}) 
        top_platform_loc = top_platform_NA.sort_values(by='Name', ascending=False).iloc[0]['Name']
        top_platform_NA = top_platform_NA[top_platform_NA['Name'] == top_platform_loc].Platform.values                  
        return top_platform_NA


    @task()
    def a_kosheleva_top_JP_publisher(data):
        top_JP_publisher = data.groupby('Publisher', as_index = False) \
                                .agg({'JP_Sales':'sum'}) 
        top_JPsales_loc = top_JP_publisher.sort_values('JP_Sales', ascending = False).iloc[0]['JP_Sales']
        top_JP_publisher = top_JP_publisher[top_JP_publisher['JP_Sales'] == top_JPsales_loc].Publisher.values
        return top_JP_publisher


    @task()
    def a_kosheleva_eu_more_than_jp_sales(data):
        eu_more_than_jp = data.query('EU_Sales > JP_Sales')['Name'].nunique()
        return eu_more_than_jp


    @task()
    def a_kosheleva_print_data(top_game, top_genre, top_platform_NA, top_JP_publisher, eu_more_than_jp):

        context = get_current_context()
        date = context['ds']

        print(f'''Game sales data in {year} for {date}:
        TOP-game according to global sales:
            {top_game}
        TOP genre in Europe:
            {top_genre}
        TOP Platform in North America with over 1 million game copies sold:
            {top_platform_NA}
        TOP Publisher in Japan (Highest Average Game Sales):
            {top_JP_publisher}
        How many games were sold better in Europe than in Japan:
            {eu_more_than_jp}''')
        

    data = a_kosheleva_get_data()
    top_game = a_kosheleva_top_game(data)
    top_genre = a_kosheleva_top_genre(data)
    top_platform_NA = a_kosheleva_top_platform_in_NA(data)
    top_JP_publisher = a_kosheleva_top_JP_publisher(data)
    eu_more_than_jp = a_kosheleva_eu_more_than_jp_sales(data)
    a_kosheleva_print_data(top_game, top_genre, top_platform_NA, top_JP_publisher, eu_more_than_jp)
        
a_kosheleva_lesson3 = a_kosheleva_lesson3()
